# Stimulus playground
This script is a playground to inspect and manipulate stimuli for a binocular rivalry (BR) experiment<br>
In this experiment (as at April th) I aim to present objects that are associated with a color (e.g., banana - yellow, strawberry - red) in rivarous situations.<br>
1. Show the same stimulus either in the "correct" and in an "incorrect" color (a banana in yellow and blue)
2. Show two different stimuli that are associated with different colors but in the same color (a banana and a strawberry both in yellow)

Compare the onset and sustained dominance with rivaling gratings that follow a series of non-rivaling gratings that appear to be rotating (Denison et al., 2011, Attarha et al., 2015)

## Stimuli
In the following I will look at the stimuli used by Teichmann et al., 2020

In [1]:
# import libraries

# use the os
import os
import glob
# math and data structure
import numpy as np
import pandas as pd
# plotting
import matplotlib.pyplot as plt
# image processing
import cv2

In [73]:
# stimuli from Teichmann et al., 2020
# provided by her OSF site (https://osf.io/tcqjh/)
stim_dir = os.path.join('..','stimuli')
original_stim_dir = os.path.join(stim_dir,'original')
true_color_stim_dir = os.path.join(stim_dir,'true_color2')
inverted_color_stim_dir = os.path.join(stim_dir,'inverted2')
grey_stim_dir = os.path.join(stim_dir,'grey') # not used yet
# get a list of all stimuli in the directory
stimuli_full_path = glob.glob(os.path.join(original_stim_dir,'*.png'))
stimuli_full_path.sort()

# extract the image name from the stimulus path
stimuli = [os.path.basename(stim) for stim in stimuli_full_path]

In [92]:
# read out the original stimuli and remove the background
for stim_path, stim in zip(stimuli_full_path, stimuli):
    # output directories
    true_color_output_dir = os.path.join(true_color_stim_dir, stim)
    
    # read in the original image
    img = cv2.imread(stim_path)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image to isolate the non-white pixels
    _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

    # Find the contours in the thresholded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour with the largest area
    largest_contour = max(contours, key=cv2.contourArea)

    # Create a binary mask for the largest contour
    mask = np.zeros(gray.shape, dtype=np.uint8)
    cv2.drawContours(mask, [largest_contour], -1, (255, 255, 255), cv2.FILLED)

    # Create a binary mask with the pixels inside the contour set to white
    # and the pixels outside the contour set to black
    binary_mask = cv2.inRange(mask, 255, 255)
    
    # convert image from BGR to HLS space
    hls_img = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    
    # The contour also captures empty space surrounded by object boundaries
    # We therefore add a mask that includes all pixels with a luminance value of 255 -> white
    white_mask = hls_img[:,:,1]==255
    big_mask = np.logical_or(white_mask, ~binary_mask)
    
    # add alpha channel to original image by converting from BGR to BGRA space
    bgra_img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
    
    # change alpha channel of all pixels that are white to completely transparent
    # CAUTION this results in pixels that are part of the stimulus to be transparent. 
    bgra_img[big_mask == 1] -= np.array([0,0,0,255],dtype='uint8') 
    cv2.imwrite(true_color_output_dir,bgra_img)

## Invert the color in the stimuli
This is not performed in the same step because some minor adjustments were made in the true color stimuli using GIMP

In [96]:
for stim in stimuli:
    # output directories
    true_color_input_dir = os.path.join(true_color_stim_dir, stim)
    inverted_color_output_dir = os.path.join(inverted_color_stim_dir, stim)
    
    # read in the original image
    img = cv2.imread(true_color_input_dir)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image to isolate the non-white pixels
    _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

    # Find the contours in the thresholded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour with the largest area
    largest_contour = max(contours, key=cv2.contourArea)

    # Create a binary mask for the largest contour
    mask = np.zeros(gray.shape, dtype=np.uint8)
    cv2.drawContours(mask, [largest_contour], -1, (255, 255, 255), cv2.FILLED)

    # Create a binary mask with the pixels inside the contour set to white
    # and the pixels outside the contour set to black
    binary_mask = cv2.inRange(mask, 255, 255)
    
    # convert image from BGR to HLS space
    hls_img = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    
    # The contour also captures empty space surrounded by object boundaries
    # We therefore add a mask that includes all pixels with a luminance value of 255 -> white
    white_mask = hls_img[:,:,1]==255
    big_mask = np.logical_or(white_mask, ~binary_mask)
    # create a copy of the hls_img and invert the hue
    inverted_hls_img = np.copy(hls_img)
    # select all pixels that are not white and increase the hue value by 90 (hue ranges from 0-179)
    inverted_hls_img[np.where(binary_mask != 0)]+=np.array([90,0,0],dtype='uint8')
    # convert inverted hls image back into BGR 
    inverted_bgr_img = cv2.cvtColor(inverted_hls_img, cv2.COLOR_HLS2BGR)
    
    # add alpha channel to inverted image by converting from BGR to BGRA space
    inverted_bgra_img = cv2.cvtColor(inverted_bgr_img, cv2.COLOR_BGR2BGRA)

    inverted_bgra_img[big_mask == 1] -= np.array([0,0,0,255],dtype='uint8')
    cv2.imwrite(inverted_color_output_dir, inverted_bgra_img)